In [30]:
import pandas as pd 
import numpy as np
import openpyxl
import os 
from pathlib import Path
import shutil

In [31]:
# CHANGE THIS FILE PATH
file_path = "C:/Users/jordancook/OneDrive - Texas A&M University/TAMU Grad School/Research/EIA Data/Raw Data/"
year = '2023/'

In [32]:
data = '3_1_Generator_Y2023.xlsx'
file = file_path + year + data
gen = pd.read_excel(file, sheet_name='Proposed', skiprows=1)

In [33]:
gen = gen.drop(gen.index[-1])

gen = gen.rename(columns={'Plant Code': 'Number of Bus'})
gen = gen.rename(columns={'State': 'Zone Name'})

col_name = 'ID'
gen.insert(1, col_name, None)

gen = gen.sort_values(by='Number of Bus')

col_name = 'Primary Label' 
gen.insert(2, col_name, None)

gen['Number of Bus'] = gen['Number of Bus'].astype(int)
gen['Primary Label'] = gen['Number of Bus'].astype(str) + '_' + gen['Generator ID'].astype(str)

gen['ID'] = gen.index % 100

gen = gen.rename(columns={'Prime Mover': 'Unit Type Code'})
gen = gen.rename(columns={'Energy Source 1': 'Fuel Type Code'})
gen = gen.rename(columns={'Effective Month': 'Cust Int 3'})
gen = gen.rename(columns={'Effective Year': 'Cust Int 4'})
gen = gen.rename(columns={'Status': 'Cust String 3'})

gen['Min MW'] = 0
gen['Status'] = 'Yes'
gen['Set Volt '] = 1.0 
gen['Gen MW set point'] = 0 
gen['Max Mvar'] = 0 
gen['Min Mvar'] = 0 
gen['Gen Mvar set point'] = 0 
gen['AGC'] = 'No'
gen['AVR'] = 'No'
gen['MVA Base'] = gen['Nameplate Capacity (MW)']
gen['RegBus Num'] = gen['Number of Bus']
gen['Net Summer Capacity (MW)'] = pd.to_numeric(gen['Summer Capacity (MW)'], errors='coerce')
gen['Net Winter Capacity (MW)'] = pd.to_numeric(gen['Winter Capacity (MW)'], errors='coerce')
gen['Max MW'] = np.maximum(gen['Net Summer Capacity (MW)'], gen['Net Winter Capacity (MW)'])
gen['EIA860 Gen ID'] = gen['Generator ID']
gen['EIA860 Plant Code'] = gen['Number of Bus']

gen.loc[gen['Max MW'].isna(), 'Max MW'] = 0
gen[gen['Max MW'].isna()]

gen = gen.rename(columns={'Current Month': 'Operating Date Month'})
gen = gen.rename(columns={'Current Year': 'Operating Date Year'})

columns_to_delete = [
    'Associated with Combined Heat and Power System',
    'Can Bypass Heat Recovery Steam Generator?',
    'Carbon Capture Technology?',
    'Cofire Fuels?',
    'County',
    'Duct Burners',
    'Energy Source 2',
    'Energy Source 3',
    'Energy Source 4',
    'Energy Source 5',
    'Energy Source 6',
    'Fluidized Bed Technology?',
    'Multiple Fuels?',
    'Nameplate Capacity (MW)',
    'Nameplate Power Factor',
    'Net Summer Capacity (MW)',
    'Net Winter Capacity (MW)',
    'Other Combustion Technology?',
    'Ownership',
    'Plant Name', 
    'Previously Canceled',
    'Pulverized Coal Technology?',
    'RTO/ISO LMP Node Designation',
    'RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC',
    'Sector',
    'Sector Name',
    'Solid Fuel Gasification System?',
    'Stoker Technology?',
    'Subcritical Technology?',
    'Summer Capacity (MW)',
    'Supercritical Technology?',
    'Switch Between Oil and Natural Gas?',
    'Technology',
    'Turbines or Hydrokinetic Buoys',
    'Ultrasupercritical Technology?',
    'Unit Code',
    'Utility Name',
    'Winter Capacity (MW)'
]
gen = gen.drop(columns=columns_to_delete)

gen.to_excel('Gens_future.xlsx', index=False)

#Now adding in the Gen at the top to paste in
workbook = openpyxl.load_workbook('Gens_future.xlsx')
sheet = workbook.worksheets[0]
sheet.insert_rows(1)
sheet['A1'] = 'Gen'
workbook.save('Gens_future.xlsx')

In [34]:
# Putting all the data in the right folder
# CHANGE THESE PATHS
source_directory = Path("C:/Users/jordancook/OneDrive - Texas A&M University/TAMU Grad School/Research/EIA Data/Automation Code")
folder_path = Path("C:/Users/jordancook/OneDrive - Texas A&M University/TAMU Grad School/Research/EIA Data/Automation Code/2023")

# Create the folder if it doesn't exist
folder_path.mkdir(parents=True, exist_ok=True)

# Move all xlsx files from the source directory to the target folder
for xlsx_file in source_directory.glob("*.xlsx"):
    shutil.move(str(xlsx_file), str(folder_path / xlsx_file.name))